# Object Detection Pipeline - Hyperparameter Tuning & Model Training

Hyperparameter tuning with Ray Tune on validation set, followed by final training on train+val and evaluation on test set.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kshitijrajsharma/dl4cv-oda/blob/master/notebooks/experiment.ipynb)

In [1]:
# ! pip install dl4cv_oda ray[tune]

In [2]:
import os
import yaml
import shutil
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from dl4cv_oda import (clean_osm_data, clip_labels_to_tiles, convert_to_yolo_format,
                       create_train_val_split, create_yolo_config, download_tiles)
from ultralytics import YOLO, RTDETR

## Configuration

In [3]:
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
CHIPS_DIR = DATA_DIR / "chips"
LABELS_DIR = DATA_DIR / "labels"
YOLO_DIR = DATA_DIR / "yolo"

OSM_FILE = RAW_DIR / "kolovai-trees.geojson"
CLEANED_FILE = RAW_DIR / "cleaned.geojson"
TREES_BOX_FILE = DATA_DIR / "trees_box.geojson"
TILES_FILE = DATA_DIR / "tiles.geojson"

IMG_SIZE = 256
EPOCHS = 50
PATIENCE = 10

TUNE_EPOCHS = 10
TUNE_ITERATIONS = 2
GPU_PER_TRIAL = 1

MODELS_TO_TRAIN = {
    "yolov8l": "yolov8l.pt",
    "yolo12l": "yolo12l.pt",
    "rtdetr-l": "rtdetr-l.pt"
}

In [4]:
import requests
if not OSM_FILE.exists():
    OSM_FILE.parent.mkdir(parents=True, exist_ok=True)
    OSM_FILE.write_bytes(requests.get("https://github.com/kshitijrajsharma/dl4cv-oda/blob/master/data/raw/kolovai-trees.geojson?raw=true", allow_redirects=True).content)
    print(f"Downloaded {OSM_FILE}")
else:
    print("OSM data ready")

OSM data ready


## Step 1: Clean OSM Data

In [5]:
if not CLEANED_FILE.exists():
    count = clean_osm_data(str(OSM_FILE), str(CLEANED_FILE), str(TREES_BOX_FILE))
    print(f"Processed {count} trees")
else:
    print("Cleaned data ready")

Cleaned data ready


## Download Tiles

In [6]:
if not TILES_FILE.exists():
    data = gpd.read_file(TREES_BOX_FILE)
    data.to_crs(epsg=4326, inplace=True)
    bbox = list(data.total_bounds)
    await download_tiles(bbox, 19, "https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a175-9b667174f48c/{z}/{x}/{y}.png", DATA_DIR, 'OAM')
    print("Tiles downloaded")

## Clip Labels & Convert to YOLO Format

In [7]:
if TILES_FILE.exists() and not (YOLO_DIR / "train").exists():
    stats = clip_labels_to_tiles(str(TREES_BOX_FILE), str(TILES_FILE), str(LABELS_DIR))
    print(f"{stats['processed']} tiles, {stats['total_trees']} trees")
    class_mapping = convert_to_yolo_format(str(TREES_BOX_FILE), str(CHIPS_DIR), str(LABELS_DIR), str(YOLO_DIR), target_species="Coconut")
    print(f"Class mapping: {class_mapping}")
else:
    print("YOLO format data ready")

YOLO format data ready


## Create Train/Val/Test Split

In [8]:
if not (YOLO_DIR / "train").exists():
    train_count, val_count, test_count = create_train_val_split(str(LABELS_DIR), str(CHIPS_DIR), str(YOLO_DIR), train_ratio=0.7, val_ratio=0.2, test_ratio=0.1)
    print(f"Train: {train_count} | Val: {val_count} | Test: {test_count}")
else:
    print("Train/Val/Test split ready")

config_file = create_yolo_config(str(YOLO_DIR), {"Coconut": 0})
config_file_abs = os.path.abspath(config_file)
print(f"Config: {config_file_abs}")

Train/Val/Test split ready
Config: /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/data/yolo/config.yaml


## Hyperparameter Tuning with Ray Tune

Tune all models with consistent parameters.

In [ ]:
tune_results = {}
for model_name, model_weights in MODELS_TO_TRAIN.items():
    print(f"\n{'='*60}\nTuning {model_name}...\n{'='*60}")
    model = RTDETR(model_weights) if "rtdetr" in model_name.lower() else YOLO(model_weights)
    result_grid = model.tune(data=config_file_abs, epochs=TUNE_EPOCHS, imgsz=IMG_SIZE, 
                            patience=PATIENCE, iterations=TUNE_ITERATIONS, 
                            gpu_per_trial=GPU_PER_TRIAL, use_ray=True)
    tune_results[model_name] = result_grid
    print(f"{model_name} tuning complete")

(_tune pid=3353545) New https://pypi.org/project/ultralytics/8.3.252 available 😃 Update with 'pip install -U ultralytics'
(_tune pid=3353545) Ultralytics 8.3.250 🚀 Python-3.13.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15944MiB)
(_tune pid=3353545) engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.08291607587131566, box=0.12082037432878465, cache=False, cfg=None, classes=None, close_mosaic=10, cls=2.5001120168415434, compile=False, conf=None, copy_paste=0.7448437572121328, copy_paste_mode=flip, cos_lr=False, cutmix=0.9142083313199891, data=/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/data/yolo/config.yaml, degrees=12.202317759488318, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.1476119576937308, flipud=0.3005406119855477, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.03142193567692

(_tune pid=3353545) /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/ray/train/_internal/session.py:792: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(_tune pid=3353545)   _log_deprecation_warning(


       2/10      2.88G    0.04043      6.837      1.253        460        256: 4% ──────────── 1/28 2.1it/s 0.1s<12.8s
       2/10      2.88G    0.04094      7.278      1.268        495        256: 11% ━─────────── 3/28 6.0it/s 0.3s<4.2s
       2/10      2.88G    0.03921      6.878      1.236        457        256: 18% ━━────────── 5/28 8.6it/s 0.4s<2.7s
       2/10      2.88G    0.03921      6.878      1.236        457        256: 18% ━━────────── 5/28 8.6it/s 0.4s<2.7s
       2/10      2.88G    0.03871      6.735      1.228        414        256: 25% ━━━───────── 7/28 10.3it/s 0.6s<2.0s
       2/10      2.88G    0.03833       6.52      1.218        413        256: 32% ━━━╸──────── 9/28 11.4it/s 0.7s<1.7s
       2/10      2.88G    0.03871      6.735      1.228        414        256: 25% ━━━───────── 7/28 10.3it/s 0.6s<2.0s
       2/10      2.88G    0.03833       6.52      1.218        413        256: 32% ━━━╸──────── 9/28 11.4it/s 0.7s<1.7s
       2/10      2.88G    0.03829      6.473

(_tune pid=3353545) /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/ray/train/_internal/session.py:792: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(_tune pid=3353545)   _log_deprecation_warning(


       3/10      2.88G    0.03603      6.823      1.175        538        256: 4% ──────────── 1/28 2.2it/s 0.1s<12.1s
       3/10      2.88G    0.03464      6.778      1.157        309        256: 11% ━─────────── 3/28 5.9it/s 0.3s<4.2s
       3/10      2.88G     0.0349      6.574      1.154        278        256: 18% ━━────────── 5/28 9.4it/s 0.4s<2.4s
       3/10      2.88G     0.0349      6.574      1.154        278        256: 18% ━━────────── 5/28 9.4it/s 0.4s<2.4s
       3/10      2.88G    0.03469      6.585      1.146        338        256: 25% ━━━───────── 7/28 11.1it/s 0.5s<1.9s
       3/10      2.88G    0.03503      6.517      1.146        287        256: 32% ━━━╸──────── 9/28 12.2it/s 0.7s<1.6s
       3/10      2.88G    0.03469      6.585      1.146        338        256: 25% ━━━───────── 7/28 11.1it/s 0.5s<1.9s
       3/10      2.88G    0.03503      6.517      1.146        287        256: 32% ━━━╸──────── 9/28 12.2it/s 0.7s<1.6s
       3/10      2.88G    0.03491      6.366

...

(_tune pid=3353545) 
(_tune pid=3353545)       Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       6/10      3.06G    0.03583      6.401      1.162        381        256: 4% ──────────── 1/28 2.3it/s 0.1s<11.9s


(bundle_reservation_check_func pid=3351658) [2026-01-11 00:45:50,412 E 3351658 3351955] core_worker_process.cc:842: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
[2026-01-11 00:45:50,577 E 3342006 3351622] core_worker_process.cc:842: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14


       6/10      3.06G    0.03534      6.352       1.14        332        256: 11% ━─────────── 3/28 5.8it/s 0.3s<4.3s
       6/10      3.06G    0.03522      6.442      1.129        321        256: 18% ━━────────── 5/28 9.3it/s 0.4s<2.5s
       6/10      3.06G    0.03494      6.454      1.123        292        256: 25% ━━━───────── 7/28 10.7it/s 0.5s<2.0s
       6/10      3.06G    0.03507       6.45      1.129        293        256: 32% ━━━╸──────── 9/28 12.3it/s 0.7s<1.5s
       6/10      3.06G    0.03494      6.454      1.123        292        256: 25% ━━━───────── 7/28 10.7it/s 0.5s<2.0s
       6/10      3.06G    0.03507       6.45      1.129        293        256: 32% ━━━╸──────── 9/28 12.3it/s 0.7s<1.5s
       6/10      3.06G    0.03506      6.477       1.13        552        256: 39% ━━━━╸─────── 11/28 12.8it/s 0.8s<1.3s
       6/10      3.06G     0.0351      6.379      1.131        455        256: 46% ━━━━━╸────── 13/28 14.0it/s 0.9s<1.1s
       6/10      3.06G    0.03506      6

(_tune pid=3353545) /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/ray/train/_internal/session.py:792: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(_tune pid=3353545)   _log_deprecation_warning(


       7/10      3.06G    0.03373      5.391      1.107        374        256: 11% ━─────────── 3/28 5.7it/s 0.3s<4.4s
       7/10      3.06G     0.0344      5.401      1.111        439        256: 18% ━━────────── 5/28 9.1it/s 0.4s<2.5s
       7/10      3.06G    0.03408      5.396      1.113        406        256: 25% ━━━───────── 7/28 10.6it/s 0.5s<2.0s
       7/10      3.06G     0.0344      5.401      1.111        439        256: 18% ━━────────── 5/28 9.1it/s 0.4s<2.5s
       7/10      3.06G    0.03408      5.396      1.113        406        256: 25% ━━━───────── 7/28 10.6it/s 0.5s<2.0s
       7/10      3.06G     0.0343      5.692      1.117        288        256: 32% ━━━╸──────── 9/28 12.5it/s 0.6s<1.5s
       7/10      3.06G    0.03473      5.787      1.125        448        256: 39% ━━━━╸─────── 11/28 12.9it/s 0.8s<1.3s
       7/10      3.06G     0.0343      5.692      1.117        288        256: 32% ━━━╸──────── 9/28 12.5it/s 0.6s<1.5s
       7/10      3.06G    0.03473      5.7

## Analyze Results & Extract Best Configs

In [ ]:
def analyze_and_save_results(result_grid, model_name):
    results_list = []
    for i, result in enumerate(result_grid):
        df = result.metrics_dataframe
        p = df['metrics/precision(B)'].iloc[-1] if 'metrics/precision(B)' in df else 0
        r = df['metrics/recall(B)'].iloc[-1] if 'metrics/recall(B)' in df else 0
        m = df['metrics/mAP50(B)'].iloc[-1] if 'metrics/mAP50(B)' in df else 0
        f1 = 2 * (p * r) / (p + r + 1e-6) if (p + r) > 0 else 0
        results_list.append({'trial': i, 'config': result.config, 'precision': p, 'recall': r, 'mAP50': m, 'f1': f1})
    
    results_df = pd.DataFrame(results_list).sort_values('f1', ascending=False)
    best = results_df.iloc[0]
    best_config = best['config']
    
    with open(YOLO_DIR / f"best_config_{model_name}.yaml", 'w') as f:
        yaml.dump(best_config, f, default_flow_style=False)
    
    print(f"{model_name:12s} F1: {best['f1']:.4f} | P: {best['precision']:.4f} | R: {best['recall']:.4f} | mAP50: {best['mAP50']:.4f}")
    return results_df, best_config

best_configs = {}
for model_name, result_grid in tune_results.items():
    _, best_configs[model_name] = analyze_and_save_results(result_grid, model_name)

## Merge Train+Val for Final Training

In [ ]:
trainval_dir = YOLO_DIR / "trainval"
trainval_config = YOLO_DIR / "data_trainval.yaml"

if not trainval_dir.exists():
    trainval_dir.mkdir(exist_ok=True)
    for split in ["train", "val"]:
        for f in (YOLO_DIR / split).glob("*"):
            shutil.copy(f, trainval_dir / f.name)
    with open(config_file, 'r') as f:
        trainval_config.write_text(f.read().replace("train: train", "train: trainval"))
    print(f"Merged train+val into {trainval_dir}")
else:
    print("Train+val merge ready")

## Final Training with Best Hyperparameters

In [ ]:
trained_models = {}
for model_name, model_weights in MODELS_TO_TRAIN.items():
    print(f"\nTraining {model_name} on train+val...")
    model = RTDETR(model_weights) if "rtdetr" in model_weights.lower() else YOLO(model_weights)
    model.train(data=str(trainval_config), epochs=EPOCHS, imgsz=IMG_SIZE, patience=PATIENCE,
                name=f"final_{model_name}", plots=True, **best_configs[model_name])
    trained_models[model_name] = model
    print(f"✓ {model_name} complete")

## Evaluate on Test Set

In [ ]:
test_results = {}
for name, model in trained_models.items():
    print(f"Evaluating {name}...")
    m = model.val(split='test')
    p, r = m.box.mp, m.box.mr
    test_results[name] = {"precision": p, "recall": r, "f1": 2 * (p * r) / (p + r + 1e-6),
                         "mAP50": m.box.map50, "mAP50-95": m.box.map}

results_df = pd.DataFrame(test_results).T.sort_values('f1', ascending=False)
print("\n" + "="*60 + "\nTest Set Results:\n" + "="*60)
print(results_df.to_string())
print(f"\nBest: {results_df.index[0]}")

## F1 Score per Epoch

In [ ]:
n = len(MODELS_TO_TRAIN)
cols = min(3, n)
rows = (n + cols - 1) // cols
fig, axes = plt.subplots(rows, cols, figsize=(6*cols, 5*rows))
axes = axes.flatten() if n > 1 else [axes]

for idx, model_name in enumerate(MODELS_TO_TRAIN.keys()):
    csv = Path(f"runs/detect/final_{model_name}/results.csv")
    if csv.exists():
        df = pd.read_csv(csv)
        df.columns = df.columns.str.strip()
        p, r = df['metrics/precision(B)'], df['metrics/recall(B)']
        f1 = 2 * (p * r) / (p + r + 1e-6)
        axes[idx].plot(f1, label='F1', linewidth=2, color='#2E86AB')
        axes[idx].plot(p, label='Precision', alpha=0.7, color='#A23B72')
        axes[idx].plot(r, label='Recall', alpha=0.7, color='#F18F01')
        axes[idx].set_title(model_name.upper(), fontsize=11, fontweight='bold')
        axes[idx].set_xlabel('Epoch')
        axes[idx].set_ylabel('Score')
        axes[idx].legend()
        axes[idx].grid(True, alpha=0.3)

for idx in range(n, len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.savefig('f1_scores_comparison.png', dpi=150, bbox_inches='tight')
plt.show()